In [0]:
import numpy as np
import os
from PIL import Image
import time
import json
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import random

imageName = 'example.jpg'
TopPick = 5

In [0]:
imageFile = Image.open(imageName)
imageData = np.asarray(imageFile)

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
# Load a checkpoint and rebuild the model

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    optimizer = checkpoint['optimizer']
    epochs = checkpoint['epochs']
    
    for param in model.parameters():
        param.requires_grad = False
        
    return model, checkpoint['class_to_idx']

In [0]:
# Load the model

model, class_to_idx = load_checkpoint('checkpoint.pth')

idx_to_class = { v : k for k,v in class_to_idx.items()}

In [0]:
# Process the image to be classified

def process_image(image):
    preprocess = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
    ])
    image = preprocess(image)
    return image

In [0]:
# Show the image to be classified

def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    image = image.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    image = np.clip(image, 0, 1)
    ax.imshow(image)
    
    return ax

In [0]:
plt.imshow(imageFile)

In [0]:
# Classify the image

def predict2(image_path, model, TopPick):
    img = Image.open(image_path)
    img = process_image(img)
    img = np.expand_dims(img, 0)
    img = torch.from_numpy(img)
    model.eval()
    inputs = Variable(img).to(device)
    logits = model.forward(inputs)
    ps = F.softmax(logits,dim=1)
    topk = ps.cpu().topk(topk)
    return (e.data.numpy().squeeze().tolist() for e in topk)

In [0]:
data_transforms = {
    'Training': transforms.Compose([
        transforms.Resize(224),
        rotation_transform,
        transforms.ToTensor(),
    ]),
    'Validation': transforms.Compose([
        transforms.Resize(224) ,
        transforms.ToTensor(),
    ])
}


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), \
                                          data_transforms[x]) for x in ['Training', 'Validation']}
                                          
class_names = image_datasets['Training'].classes

In [0]:
# Generate smaller images

def imageCropping(originalImg):
  base1 = originalImg.shape[0]
  base2 = originalImg.shape[1]
  base = min(base1, base2)
  lBound = int(base * 0.3)
  uBound = int(base * 0.7)
  length1 = random.randint(lBound, uBound)
  length2 = random.randint(lBound, uBound)
  x1 = random.randint(0, base2 - length1 - 1)
  y1 = random.randint(0, base1 - length1 - 1)
  x2 = random.randint(0, base2 - length2 - 1)
  y1 = random.randint(0, base1 - length2 - 1)
  newimg1 = originalImg[y1:y1+length1, x1:x1+length1]
  newimg2 = originalImg[y2:y2+length2, x2:x2+length2]
  return newimg1, newimg2

In [0]:
identified = 0
obj1 = ''
obj2 = ''

while identified == 0:
  img1, img2 = imageCropping(imageData)
  im1 = Image.fromarray(img1)
  im2 = Image.fromarray(img2)
  im1.save('temp1.jpg','JPEG')
  im2.save('temp2.jpg','JPEG')
  probs1, classes1 = predict2('temp1.jpg', model.to(device), TopPick)
  object_names1 = [class_names[e] for e in classes1]
  probs2, classes2 = predict2('temp2.jpg', model.to(device), TopPick)
  object_names2 = [class_names[e] for e in classes2]
  if probs1[0] > 0.9 and probs2[0] > 0.9:
    identified = 1
    obj1 = object_names1[0]
    obj2 = object_names2[0]

In [0]:
print(obj1, obj2)

In [0]:
f = open("identified.txt", "w")
f.write(obj1)
f.write("\n")
f.write(obj2)
f.close()